In [97]:
import re
import pandas as pd

from tqdm.auto import tqdm

In [98]:
SEED = 42

In [99]:
prompt_dataset = pd.read_csv('1M_midjourney_prompts.csv')

prompt_dataset.rename(columns={
    'Mind Manifesting poseidon in ocean, holding trident, sharks, rpg style, ': 'prompt',
    'Random, Pose, Old, Style, Mind Manifesting': 'keyword'
    },inplace=True)

In [100]:
prompt_dataset.head()

,prompt,keyword
0,Diagram Make The Starling Night paintings real...,"Random, 4K, Reality, Paint, Painting, Cinemati..."
1,"Magical Girl cute kitten wearing suit, teachin...","Random, Cartoon, Ground, Art, Bright, Style, C..."
2,"Frank Martin sleeve tattoo , wave, style of Ho...","Random, Tattoo, Sleeve, Style, Dragon, Frank M..."
3,Close Face /imagine futuristic magazine ad for...,"Random, Zine, Futuristic, Pan, Close Face"
4,Otaku wide shot of a huge red giant star in sp...,"Random, In The Distance, Silhouette, Spaceship..."


In [101]:
len(prompt_dataset)

999999

In [102]:
def get_keywords(keyword):
    return [k.strip().lower() for k in keyword.split(', ') if k != 'Random']

In [103]:
prompt_dataset['option'] = prompt_dataset['keyword'].apply(lambda x: get_keywords(x))
prompt_dataset.drop(['keyword'], axis=1, inplace=True)

In [104]:
prompt_dataset.head()

,prompt,option
0,Diagram Make The Starling Night paintings real...,"[4k, reality, paint, painting, cinematic, wood..."
1,"Magical Girl cute kitten wearing suit, teachin...","[cartoon, ground, art, bright, style, cute, cu..."
2,"Frank Martin sleeve tattoo , wave, style of Ho...","[tattoo, sleeve, style, dragon, frank martin]"
3,Close Face /imagine futuristic magazine ad for...,"[zine, futuristic, pan, close face]"
4,Otaku wide shot of a huge red giant star in sp...,"[in the distance, silhouette, spaceship, wide ..."


In [106]:
for i, row in prompt_dataset.iterrows():
    if i > 3: break

    print('Prompt:', row['prompt'])
    print('Option:', row['option'])
    
    print('\n')

Prompt: Diagram Make The Starling Night paintings reality, in the woods, 4k , cinematic, 
Option: ['4k', 'reality', 'paint', 'painting', 'cinematic', 'wood', 'diagram']


Prompt: Magical Girl cute kitten wearing suit, teaching in class room, bright light, clear background, cartoon style, v5, 
Option: ['cartoon', 'ground', 'art', 'bright', 'style', 'cute', 'cut', 'background', 'magical girl']


Prompt: Frank Martin sleeve tattoo , wave, style of Hokusai, cherry blossoms, setting sun, D&d dragon, Caffeine molecular compound, 
Option: ['tattoo', 'sleeve', 'style', 'dragon', 'frank martin']


Prompt: Close Face /imagine futuristic magazine ad for a tech company called Moon, 
Option: ['zine', 'futuristic', 'pan', 'close face']




In [108]:
option_dict = {}

for i, row in tqdm(prompt_dataset.iterrows()):
    for k in row['option']:
        if k in option_dict:
            option_dict[k] += 1
            
        else:
            option_dict[k] = 1

999999it [00:29, 33805.27it/s]


In [109]:
keyword_dict = sorted(option_dict.items(), key=lambda item: item[1], reverse=True)

In [112]:
keyword_dict

[('style', 210074),
 ('photo', 203161),
 ('realistic', 197977),
 ('art', 194262),
 ('detail', 186768),
 ('color', 176487),
 ('ground', 166001),
 ('hit', 162320),
 ('background', 153602),
 ('up', 134966),
 ('detailed', 128034),
 ('design', 126131),
 ('old', 124156),
 ('lighting', 100631),
 ('illustration', 100569),
 ('photograph', 98484),
 ('8k', 90291),
 ('cinematic', 86418),
 ('beautiful', 77631),
 ('photography', 68498),
 ('shot', 68493),
 ('hair', 68438),
 ('era', 68424),
 ('dark', 66058),
 ('line', 65633),
 ('paint', 58852),
 ('photorealistic', 57904),
 ('white background', 51997),
 ('details', 51076),
 ('tree', 49815),
 ('image', 48750),
 ('pen', 46140),
 ('lens', 46075),
 ('painting', 45188),
 ('ink', 44867),
 ('logo', 44493),
 ('ray', 44221),
 ('cut', 43495),
 ('pan', 43488),
 ('drawing', 43337),
 ('portrait', 42937),
 ('space', 42688),
 ('resolution', 42000),
 ('soft', 41206),
 ('eyes', 40960),
 ('raw', 40468),
 ('scene', 39815),
 ('fantasy', 38034),
 ('rough', 37521),
 ('4k', 

In [113]:
len(option_dict)

2606

In [115]:
from datasets import Dataset
prompt_dataset = Dataset.from_pandas(prompt_dataset)

prompt_dataset = prompt_dataset.train_test_split(test_size=0.1, seed=SEED)

# Split the train set further into train and validation sets (90% train, 10% validation)
prompt_dataset['train'] = prompt_dataset['train'].train_test_split(test_size=0.1)

# Access the resulting datasets
train_dataset = prompt_dataset['train']['train'].to_pandas()
valid_dataset = prompt_dataset['train']['test'].to_pandas()
test_dataset = prompt_dataset['test'].to_pandas()

In [116]:
train_dataset.to_csv('train.csv', index=False)
valid_dataset.to_csv('valid.csv', index=False)
test_dataset.to_csv('test.csv', index=False)

In [117]:
print(len(train_dataset), len(valid_dataset), len(test_dataset))

809999 90000 100000
